# Imports

In [1]:
from LC_SNN import LC_SNN, load_LC_SNN

# Creating network

In [ ]:
net = LC_SNN(10000)

# Training network

In [ ]:
net.train(n_iter=5000, vis_interval=60)

# View network weights

In [ ]:
net.plot_weights_XY().show()

# Calibrating network

In [ ]:
net.calibrate(10000)

# Network accuracy

In [ ]:
accuracy = net.accuracy(10000)
print(f'Network accuracy is {accuracy}')

# Saving network

In [ ]:
net.save(r'networks/example_MNIST')

# Loading network

In [ ]:
net = load_LC_SNN(r'networks/example_MNIST')

# The end

In [4]:
net = load_LC_SNN(r'networks/base_25')

In [25]:
import torch

tensor([], size=(0, 1), dtype=torch.int64)